### Dùng điều kiện số nguyên ngay từ đầu

In [25]:
from pulp import *

# Tạo bài toán
problem = LpProblem("TuyenNhanVien", LpMinimize)

# Khai báo biến: 5 biến x1, x2, x3, x4, x5
items = range(5)  
variables = LpVariable.dicts(name="CaLamViec", indices=items, lowBound=0, cat=LpInteger) 

# Hàm mục tiêu
problem += lpSum(variables[i] * [230, 250, 280, 250, 300][i] for i in items)

# Ràng buộc
problem += lpSum(variables[i] * [1, 0, 0, 1, 0][i] for i in items) >= 9
problem += lpSum(variables[i] * [1, 1, 0, 0, 1][i] for i in items) >= 20
problem += lpSum(variables[i] * [0, 1, 1, 1, 0][i] for i in items) >= 11
problem += lpSum(variables[i] * [0, 0, 1, 0, 1][i] for i in items) >= 17

# Giải bài toán
problem.solve()

# In phương án tối ưu
print("Phương án tối ưu: (", end="")
for i in range(0, items.__len__()-1):
    print(int(variables[i].value()), end=", ")
print(int(variables[items.__len__()-1].value()), end=")\n")
print("=====================================")
print("Số nhân viên làm việc ca sáng + trưa: ", int(variables[0].value()))
print("Số nhân viên làm việc ca trưa + chiều: ", int(variables[1].value()))
print("Số nhân viên làm việc ca chiều + tối: ", int(variables[2].value()))
print("Số nhân viên làm việc ca sáng + chiều: ", int(variables[3].value()))
print("Số nhân viên làm việc ca trưa + tối: ", int(variables[4].value()))
print("=====================================")
# In chi phí tối thiểu
print("Tổng chi phí tối thiểu:", int(value(problem.objective)), end="k")

Phương án tối ưu: (10, 2, 9, 0, 8)
Số nhân viên làm việc ca sáng + trưa:  10
Số nhân viên làm việc ca trưa + chiều:  2
Số nhân viên làm việc ca chiều + tối:  9
Số nhân viên làm việc ca sáng + chiều:  0
Số nhân viên làm việc ca trưa + tối:  8
Tổng chi phí tối thiểu: 7720k

### Giải với số thực rồi điều chỉnh thành số nguyên dần dần thông qua phương pháp nhánh cận

In [23]:
import scipy 
import numpy as np 

def is_integer(n):
    """Kiểm tra một số có phải là số nguyên hay không"""
    if int(n) == n:
        return True
    else:
        return False

def branch_and_bound(c, A, b, integer_var, lb, ub, isMax = False, depth=1):
    """
    Giải bài toán quy hoạch tuyến tính nguyên bằng phương pháp nhánh cận

    Args:
        c: Hệ số mục tiêu (dạng numpy array)
        A: Ma trận hệ số vế trái của các ràng buộc (dạng numpy array)
        b: Số ở vế phải của các ràng buộc (dạng numpy array)
        integer_var: Danh sách các biến cần là số nguyên (dạng numpy array boolean)
        lb: Giới hạn dưới cho các biến (dạng numpy array)
        ub: Giới hạn trên cho các biến (dạng numpy array)
        isMax: Hàm mục tiêu tìm max hay min (mặc định False - tìm min)
        depth: Độ sâu của nhánh (dùng để theo dõi quá trình đệ quy)

    Returns:
        x: Nghiệm khả thi (dạng numpy array)
        f: Giá trị mục tiêu tại nghiệm (float)
        depth: Độ sâu của nhánh tìm thấy nghiệm
    """
    
    # Giải bài toán quy hoạch tuyến tính (LP relaxation) để lấy nghiệm ước lượng
    if(isMax):
        optimized = scipy.optimize.linprog(-c, A, b, method="highs", bounds=list(zip(lb, ub)))
    else:
        optimized = scipy.optimize.linprog(c, A, b, method="highs", bounds=list(zip(lb, ub)))

    x_candidate = optimized.x # Nghiệm ước lượng
    f_candidate = optimized.fun # Giá trị mục tiêu tại nghiệm ước lượng

    print(f"Depth: {depth}, x: {x_candidate}, f: {f_candidate}")
    
    var_constraint_fulfilled = True # Biến kiểm tra các biến có thỏa mãn là số nguyên hay không

    # Nếu không tìm thấy nghiệm ước lượng thì trả về giá trị không thể đạt được
    if(f_candidate == None): 
        if(isMax):
            return [], -np.inf, depth
        else:
            return [], np.inf, depth
            
    # Kiểm tra xem nghiệm ước lượng có thỏa mãn các biến phải là số nguyên hay không
    for idx, bool in enumerate(integer_var):
        if(bool and not is_integer(x_candidate[idx])):
            var_constraint_fulfilled = False 
            break
        else:
            var_constraint_fulfilled = True

    # Nếu nghiệm ước lượng thỏa mãn thì trả về luôn    
    if(var_constraint_fulfilled): 
        if(isMax):
            return x_candidate, -f_candidate, depth
        else: 
            return x_candidate, f_candidate, depth
        
    # Nếu nghiệm ước lượng không thỏa mãn, tiến hành nhánh
    else: 
        left_lb = np.copy(lb)
        left_ub = np.copy(ub)

        right_lb = np.copy(lb)
        right_ub = np.copy(ub)

        max_coeff_idx = -1 # Chỉ số của biến có hệ số mục tiêu lớn nhất để nhánh
        for idx, val in enumerate(integer_var):  # Tìm biến không nguyên có hệ số lớn nhất
            if(val and not is_integer(x_candidate[idx])):
                if(max_coeff_idx == -1):
                    max_coeff_idx = idx
                elif(c[max_coeff_idx] < c[idx]):
                    max_coeff_idx = idx
                    
        # Tạo nhánh trái: giới hạn trên của biến được chọn bằng phần nguyên của nghiệm ước lượng
        left_ub[max_coeff_idx] = np.floor(x_candidate[max_coeff_idx])
        # Tạo nhánh phải: giới hạn dưới của biến được chọn bằng phần trần của nghiệm ước lượng
        right_lb[max_coeff_idx] = np.ceil(x_candidate[max_coeff_idx])
        print(f"Branching on x{max_coeff_idx+1} at {x_candidate[max_coeff_idx]}")

        # Giải đệ quy với các nhánh trái và phải
        x_left, f_left, depth_left = branch_and_bound(c, A, b, integer_var, left_lb, left_ub, isMax, depth + 1)
        x_right, f_right, depth_right = branch_and_bound(c, A, b, integer_var, right_lb, right_ub, isMax, depth + 1)

        # So sánh giữa 2 nhánh trái và phải để chọn ra nghiệm tốt nhất
        if(isMax):
            if(f_left > f_right):
                return x_left, f_left, depth_left
            else: 
                return x_right, f_right, depth_right
        else:
            if(f_left < f_right):
                return x_left, f_left, depth_left
            else: 
                return x_right, f_right, depth_right

In [24]:
# Hệ số hàm mục tiêu
c = np.array([230, 250, 280, 250, 300])

# Hệ số ràng buộc, ở đây ràng buộc ban đầu là Ax>=b, ta chuyển về Ax<=b bằng cách nhân -1 cho cả 2 vế
A = np.array([[-1, 0, 0, -1, 0], [-1, -1, 0, 0, -1], [0, -1, -1, -1, 0], [0, 0, -1, 0, -1]])
b = np.array([-9, -20, -11, -17])

# Các biến phải là số nguyên
integer_var = [True, True, True, True]

# Giới hạn dưới cho mỗi biến
lb = [0, 0, 0, 0, 0]
# Giới hạn trên cho mỗi biến
ub = [None, None, None , None, None]

x_optimal, f_optimal, depth_optimal = branch_and_bound(c, A, b, integer_var, lb, ub, False)
print("=====================================")

print("Phương án tối ưu: (", end="")
for i in range(len(x_optimal)-1): 
    print(int(x_optimal[i]), end=", ")
print(int(x_optimal[-1]), end=")\n")
print("=====================================")
print("Số nhân viên làm việc ca sáng + trưa: ", int(x_optimal[0]))
print("Số nhân viên làm việc ca trưa + chiều: ", int(x_optimal[1]))
print("Số nhân viên làm việc ca chiều + tối: ", int(x_optimal[2]))
print("Số nhân viên làm việc ca sáng + chiều: ", int(x_optimal[3]))
print("Số nhân viên làm việc ca trưa + tối: ", int(x_optimal[4]))
print("=====================================")
print(f"Tổng chi phí tối thiểu: {int(f_optimal)}k")
print("=====================================")
print(f"Tree depth: {depth_optimal}")

Depth: 1, x: [9.  2.5 8.5 0.  8.5], f: 7625.0
Branching on x3 at 8.5
Depth: 2, x: [8.5 2.5 8.  0.5 9. ], f: 7645.0
Branching on x2 at 2.5
Depth: 3, x: [9. 2. 8. 1. 9.], f: 7760.0
Depth: 3, x: [9. 3. 8. 0. 9.], f: 7760.0
Depth: 2, x: [10.  2.  9.  0.  8.], f: 7720.0
Phương án tối ưu: (10, 2, 9, 0, 8)
Số nhân viên làm việc ca sáng + trưa:  10
Số nhân viên làm việc ca trưa + chiều:  2
Số nhân viên làm việc ca chiều + tối:  9
Số nhân viên làm việc ca sáng + chiều:  0
Số nhân viên làm việc ca trưa + tối:  8
Tổng chi phí tối thiểu: 7720k
Tree depth: 2
